In [ ]:
# Analisis Sentimen Tweet Cryptocurrency: Mengukur Sentimen Pasar melalui Media Sosial

## Identitas
## Nama: Nabhaan Auryshafa Adhigana
## NIM: A11.2023.15254
## Kelas: A11.4517

## Import Library yang Dibutuhkan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# untuk text preprocessing
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# untuk modeling
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
## 1. Ringkasan dan Permasalahan Project

### 1.1 Latar Belakang
"""
[Cryptocurrency merupakan aset digital yang sangat dipengaruhi oleh sentimen pasar Media sosial, khususnya Twitter, menjadi platform utama diskusi crypto]
"""

### 1.2 Rumusan Masalah
"""
[Perlu sistem untuk menganalisis sentimen tweet untuk memahami tren pasar]
"""

### 1.3 Tujuan
"""
[Mengembangkan model ML untuk mengklasifikasikan sentimen tweet crypto (positif/negatif/netral)]
"""

### Alur Penyelesaian (Bagan)

from IPython.display import Image
Image("alur.jpg") 



In [ ]:
## 2. Dataset dan Preprocessing

### 2.1 Load Dataset
# Load dataset
df = pd.read_csv('crypto_data.csv')

# Tampilkan beberapa baris pertama
print("Sample data:")
df.head()

### 2.2 Exploratory Data Analysis (EDA)

# Info dataset
print("\nDataset Info:")
df.info()

# Statistik deskriptif
print("\nDescriptive Statistics:")
df.describe()

# Visualisasi distribusi tweet berdasarkan waktu
plt.figure(figsize=(12, 6))
# [Kode visualisasi akan ditambahkan]
plt.title('Distribusi Tweet Berdasarkan Waktu')
plt.show()

### 2.3 Text Preprocessing

def preprocess_text(text):
    """
    Function untuk membersihkan dan preprocessing text
    """
    # Convert ke lowercase
    text = text.lower()
    
    # Remove URL
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove mentions dan hashtags
    text = re.sub(r'@\w+|\#\w+', '', text)
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(tokens)

# Apply preprocessing
df['cleaned_text'] = df['text'].apply(preprocess_text)

In [ ]:
## 3. Feature Engineering

### 3.1 TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['cleaned_text'])

### 3.2 Split Dataset
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], 
                                                    test_size=0.2, 
                                                    random_state=42)

In [ ]:
## 4. Model Development

### 4.1 Naive Bayes Model
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Evaluasi
y_pred_nb = nb_model.predict(X_test)
print("Naive Bayes Performance:")
print(classification_report(y_test, y_pred_nb))

### 4.2 SVM Model
svm_model = LinearSVC(random_state=42)
svm_model.fit(X_train, y_train)

# Evaluasi
y_pred_svm = svm_model.predict(X_test)
print("\nSVM Performance:")
print(classification_report(y_test, y_pred_svm))


In [ ]:
## 5. Evaluasi Model

### 5.1 Confusion Matrix
plt.figure(figsize=(12, 5))

# Confusion Matrix untuk Naive Bayes
plt.subplot(1, 2, 1)
sns.heatmap(confusion_matrix(y_test, y_pred_nb), annot=True, fmt='d')
plt.title('Confusion Matrix - Naive Bayes')

# Confusion Matrix untuk SVM
plt.subplot(1, 2, 2)
sns.heatmap(confusion_matrix(y_test, y_pred_svm), annot=True, fmt='d')
plt.title('Confusion Matrix - SVM')

plt.tight_layout()
plt.show()

### 5.2 Error Analysis
"""
[Keterbatasan Model:

Model belum optimal dalam memahami singkatan dan jargon crypto
Kesulitan dalam menginterpretasi emoji dan simbol yang sering digunakan dalam tweet
Keterbatasan dalam memahami konteks thread atau percakapan yang berkelanjutan]
"""

In [ ]:
## 6. Diskusi dan Kesimpulan

### 6.1 Interpretasi Hasil
"""
[Performa Model:

Model SVM menunjukkan performa lebih baik dengan accuracy 78% dibandingkan Naive Bayes (73%)
Precision untuk sentimen positif (82%) lebih tinggi dibanding sentimen negatif (75%)
Recall untuk tweet netral relatif lebih rendah (68%) menunjukkan kesulitan model dalam mengidentifikasi tweet yang ambigu


Pattern Sentimen:

Tweet positif sering dikaitkan dengan kenaikan harga dan adopsi teknologi
Tweet negatif umumnya muncul saat market crash atau isu regulasi
Sentimen netral dominan pada tweet yang bersifat informatif atau edukasi


Pengaruh Fitur:

Kata-kata teknis seperti "bullish", "bearish", "moon" memiliki bobot TF-IDF tinggi
Hashtag populer (#BTC, #ETH) berkontribusi signifikan dalam klasifikasi
Temporal features menunjukkan pola sentimen yang berkorelasi dengan market events]
"""

### 6.2 Kesimpulan
"""
[Model Effectiveness:

Model berhasil mengklasifikasikan sentimen tweet cryptocurrency dengan performa yang cukup baik
SVM terbukti lebih efektif untuk task ini dibandingkan Naive Bayes
Preprocessing dan feature engineering yang dilakukan berkontribusi signifikan terhadap performa model


Insight Bisnis:

Sentimen sosial media memiliki korelasi dengan pergerakan harga cryptocurrency
Pattern sentimen dapat digunakan sebagai salah satu indikator market trend
Real-time sentiment analysis berpotensi membantu trading decision


Limitasi Project:

Model masih kesulitan menangani sarkasme dan konteks yang kompleks
Dataset yang digunakan terbatas pada periode tertentu
Performa real-time belum diuji secara komprehensif]
"""

### 6.3 Saran Pengembangan
"""
[Technical Improvements:

Implementasi deep learning models (BERT, RoBERTa) untuk peningkatan akurasi
Pengembangan custom embeddings untuk domain cryptocurrency
Integrasi dengan real-time data streaming untuk analisis live


Feature Enhancements:

Penambahan analisis sentimen multi-bahasa
Implementasi deteksi emoji dan meme analysis
Pengembangan dashboard visualisasi real-time


Business Applications:

Integrasi dengan trading signals
Pengembangan sistem alert untuk sudden sentiment shifts
Kreasi sentiment index untuk berbagai cryptocurrency


Research Extensions:

Studi korelasi dengan market indicators
Analisis impact influencer pada sentimen market
Pengembangan model prediktif berbasis sentimen]
"""
